# Final Assignment - UK City Contractor Guide


### *Finding the ideal city suburb to live in while working away*

This notebook presents a machine learning solution to finding the right suburb to live in in the UK when moving to a new area for work. It will use KMeans to cluster neighbourhoods within a city of choosing. Finally a map and set of summmary indicators will be presented to interpret the clusters. Leveraging Foursquare data to provide the characterisation, we can restrict the categories returned to highlight those features of interest to the user (for example, I am interested in living near a gym so might return simply those venues belonging to 'Gym / Fitness Center' [4bf58dd8d48988d175941735] )

In [68]:
#import libraries
import pandas as pd
import numpy as np

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # required
!pip install folium
import folium # map rendering library

##/ import

I have created a lookup file which maps small UK areas known as 'wards' to their 'disrict' and calculated the average latitude and longitide across UK zip/postal codes in that ward. The user will select a 'District' and will get a ward-level analyisis of their chosen area.

In [69]:
#import and show ward level lookup data
ward_url = 'https://raw.githubusercontent.com/alho9000/learning-stuff/master/q_ward_geo.csv'
wards = pd.read_csv(ward_url, error_bad_lines=False, encoding='latin-1')
wards

,District,Ward,Ward Code,Ward_Lat,Ward_Long
0,Aberdeen City,Airyhall/Broomhill/Garthdee,S13002845,57.128142,-2.135060
1,Aberdeen City,Bridge of Don,S13002836,57.189183,-2.108184
2,Aberdeen City,Dyce/Bucksburn/Danestone,S13002835,57.190119,-2.181688
3,Aberdeen City,George St/Harbour,S13002842,57.151948,-2.096323
4,Aberdeen City,Hazlehead/Queens Cross/Countesswells,S13002844,57.140356,-2.144863
5,Aberdeen City,Hilton/Woodside/Stockethill,S13002839,57.164634,-2.130853
6,Aberdeen City,Kincorth/Nigg/Cove,S13002847,57.111784,-2.096956
7,Aberdeen City,Kingswells/Sheddocksley/Summerhill,S13002837,57.153801,-2.181105
8,Aberdeen City,Lower Deeside,S13002843,57.111733,-2.222706
9,Aberdeen City,Midstocket/Rosemount,S13002841,57.150804,-2.123163


## Change the cell below to alter the location used in analysis

In [70]:
#a user can easily isolate the area of their chosing by updating the entry below

myDistrict = 'Warwick'

district = wards[wards['District']==myDistrict]
district

,District,Ward,Ward Code,Ward_Lat,Ward_Long
8138,Warwick,Bishop's Tachbrook,E05012615,52.256554,-1.547717
8139,Warwick,Budbrooke,E05012616,52.274772,-1.626225
8140,Warwick,Cubbington & Leek Wootton,E05012617,52.336649,-1.512039
8141,Warwick,Kenilworth Abbey & Arden,E05012618,52.337111,-1.675754
8142,Warwick,Kenilworth Park Hill,E05012619,52.349163,-1.564140
8143,Warwick,Kenilworth St John's,E05012620,52.339397,-1.577172
8144,Warwick,Leamington Brunswick,E05012621,52.278904,-1.532421
8145,Warwick,Leamington Clarendon,E05012622,52.291845,-1.534995
8146,Warwick,Leamington Lillington,E05012623,52.305031,-1.519627
8147,Warwick,Leamington Milverton,E05012624,52.296762,-1.545154


In [71]:
# If you can't find the District you want, try using this search tool

mySearch = 'Warwick' #change me

allDistricts = wards.groupby(by='District').count()
allDistricts = allDistricts[['Ward']].reset_index()
allDistricts.columns=['District','No Wards']
allDistricts = allDistricts[allDistricts['District'].str.contains(mySearch)]
allDistricts

,District,No Wards
233,North Warwickshire,17
351,Warwick,17


In [72]:
#to keep the user experience dynamic, we will want to generate a map which focusses initially on the center of the area of interest

#take average lat and long from district df

myLat = district['Ward_Lat'].mean()
myLong = district['Ward_Long'].mean()
myCoords = (myLat, myLong)
print("District Latitude is {}" .format(myLat))
print("District Longitude is {}" .format(myLong))

District Latitude is 52.296099780558116
District Longitude is -1.5581070514585809


In [73]:
# @hidden_cell

CLIENT_ID = 'Z4CIAMVHL1PW33YEFLTW2FWCMZQO1GLEW1E0Z5QQN5SIROIU' # your Foursquare ID
CLIENT_SECRET = 'JD3NC5LWKKDS3ZGQ2QYGGAYD2CUVZE0MKPKCLJHXKZLQUPLU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


We have now prepared the geographical data for this project. We can move on to importing and preparing the Foursquare data

In [74]:
#lets explore the Foursquare category hierarchy to determine what we want to include in our results set

#will need to build the category table and create a dataframe from it

#create request string

categories=[]

cat_url = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}' .format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION)

cat_results_all = requests.get(cat_url).json()
cat_results_topcat = cat_results_all["response"]["categories"]


In [75]:
# Create loop to iterate through json and create a lookup dataframe
# This loop will continually dive into the json response until it hits a category with no subcategories, at which point it creates a table entry.

masterlist = pd.DataFrame(columns=['topcat','subcat','subsubcat','subsubsubcat'])
catdf=pd.DataFrame.from_dict(cat_results_topcat)
catdf=catdf["name"] #create list of top level categories

if len(catdf)==0:
    print("category list empty, check request")
else:
    for i in range(0, len(catdf)):
        topcat=catdf[i] 
        # create list of relevant subcategories
        subcatdf=pd.DataFrame.from_dict(cat_results_all["response"]["categories"][i]["categories"])
        
        if len(subcatdf)==0: # if there are no subcategories, create dummy entries for category levels beneath and create entry...
            subcat='None' 
            subsubcat='None'
            subsubsubcat='None'
            masterlist = masterlist.append(pd.Series([topcat,subcat,subsubcat,subsubsubcat], index=masterlist.columns), ignore_index=True) # create row where only topcat relevant
            
        else: #create df of next level response
            subcatdf=subcatdf[["name"]] 
            for j in range(0, len(subcatdf)):
                subcat=subcatdf.iat[j,0]
                
                #create list of relevant sub-subcategories
                subsubcatdf= pd.DataFrame.from_dict(cat_results_all["response"]["categories"][i]["categories"][j]["categories"])
                                              
                if len(subsubcatdf)==0: # if there are no subcategories, create dummy entries for category levels beneath and create entry...
                    subsubcat='None'
                    subsubsubcat='None'
                    masterlist = masterlist.append(pd.Series([topcat,subcat,subsubcat,subsubsubcat], index=masterlist.columns), ignore_index=True) # create row where only topcat & subcat relevant
                    
                else: #create df of next level response
                    subsubcatdf=subsubcatdf[["name"]] 
                    for k in range(0,len(subsubcatdf)):
                        subsubcat = subsubcatdf.iat[k,0]
                        #create a list of relevant subsubsub categories
                        subsubsubcatdf=pd.DataFrame.from_dict(cat_results_all["response"]["categories"][i]["categories"][j]["categories"][k]["categories"])
                        
                        if len(subsubsubcatdf)==0: # if there are no subcategories, create dummy entries for category levels beneath and create entry...
                                subsubsubcat='None'
                                masterlist = masterlist.append(pd.Series([topcat,subcat,subsubcat,subsubsubcat], index=masterlist.columns), ignore_index=True) # create row with only subsubsubcat missing
                        else: #create df of next level response
                            subsubsubcatdf=subsubsubcatdf[["name"]] 
                            for l in range(0,len(subsubsubcatdf)):
                                subsubsubcat=subsubsubcatdf.iat[l,0]
                                masterlist = masterlist.append(pd.Series([topcat,subcat,subsubcat,subsubsubcat], index=masterlist.columns), ignore_index=True)
                        
                        
                        
print("Table generated")


Table generated


Let's have a look at the contents of the lookup table

In [76]:
masterlist.head()

#looks pretty helpful. Venues can appear in both subcat, subsub and subsubsub categories so the relevent entries used in the clustering analysis will need to be filtered by a concatenated list of 'subcat' and 'subsubcat' entries
#within a topcat of interest. Having done this, we'll remove any redundant 'None' entries

,topcat,subcat,subsubcat,subsubsubcat
0,Arts & Entertainment,Amphitheater,None,None
1,Arts & Entertainment,Aquarium,None,None
2,Arts & Entertainment,Arcade,None,None
3,Arts & Entertainment,Art Gallery,None,None
4,Arts & Entertainment,Bowling Alley,None,None


In [77]:
# let's review the summarised completed data

top_categories = masterlist.groupby('topcat').count()
print(top_categories)


                             subcat  subsubcat  subsubsubcat
topcat                                                      
Arts & Entertainment             56         56            56
College & University             36         36            36
Event                            12         12            12
Food                            306        306           306
Nightlife Spot                   23         23            23
Outdoors & Recreation           102        102           102
Professional & Other Places      99         99            99
Residence                         5          5             5
Shop & Service                  170        170           170
Travel & Transport               49         49            49


## Change the cell below to alter venues used in analysis

In [78]:
# the category I would like to analyse is the subcategory 'Athletics & Sports'

# check out the foursquare developers guide to determine your categories of interest: 
# https://developer.foursquare.com/docs/build-with-foursquare/categories/

myCats = masterlist[masterlist['subcat']=='Athletics & Sports']

# you could change the above and filter on a topcat through to a subsubcat. You could also use AND/OR operators to get a mixed analysis

myCats

,topcat,subcat,subsubcat,subsubsubcat
433,Outdoors & Recreation,Athletics & Sports,Badminton Court,None
434,Outdoors & Recreation,Athletics & Sports,Baseball Field,None
435,Outdoors & Recreation,Athletics & Sports,Basketball Court,None
436,Outdoors & Recreation,Athletics & Sports,Bowling Green,None
437,Outdoors & Recreation,Athletics & Sports,Curling Ice,None
438,Outdoors & Recreation,Athletics & Sports,Golf Course,None
439,Outdoors & Recreation,Athletics & Sports,Golf Driving Range,None
440,Outdoors & Recreation,Athletics & Sports,Gym / Fitness Center,Boxing Gym
441,Outdoors & Recreation,Athletics & Sports,Gym / Fitness Center,Climbing Gym
442,Outdoors & Recreation,Athletics & Sports,Gym / Fitness Center,Cycle Studio


In [79]:
# I need to create a single list of venues from this myCats list which unrolls subcat, subsubcat and subsubsubcat into a single list of distinct values
# Some venues belong to higher levels of the category hierarchy than others, so include every level of the hierarchy as a potential venue category.

filterList = myCats['topcat']
filterList = filterList.append(myCats['subcat'], ignore_index=True)
filterList = filterList.append(myCats['subsubcat'], ignore_index=True)
filterList = filterList.append(myCats['subsubsubcat'], ignore_index=True)
filterList = pd.DataFrame(filterList)
filterList.columns=['Selected']
filterList=filterList.groupby('Selected').count()
#remove uncategorised venues
filterList = filterList.drop(index='None')

filterList

""
Selected
Athletics & Sports
Badminton Court
Baseball Field
Basketball Court
Bowling Green
Boxing Gym
Climbing Gym
Curling Ice
Cycle Studio


Recognise the function below? It's actually been changed to use the 'search' endpoint as 'explore' asssumes you just want recommended venues and the returned json is in a different format..

In [80]:
#set initial Foursquare parameters

radius = 750 #set 500m radius from ward centre
LIMIT = 1000 #fetch 100 results only per request (ward)

#reuse the 'getNearbyVenues' function from lab

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL using 'search' endpoint:
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&intent=browse&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        #print(results) #delete
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name'] if len(v['categories'])>0 else 'None') for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [81]:
#create a new dataframe to get district venue information from foursquare

my_venues = getNearbyVenues(names=district['Ward'],
                                latitudes=district['Ward_Lat'],
                                longitudes=district['Ward_Long'])

Bishop's Tachbrook
Budbrooke
Cubbington & Leek Wootton
Kenilworth Abbey & Arden
Kenilworth Park Hill
Kenilworth St John's
Leamington Brunswick
Leamington Clarendon
Leamington Lillington
Leamington Milverton
Leamington Willes
Radford Semele
Warwick All Saints & Woodloes
Warwick Aylesford
Warwick Myton & Heathcote
Warwick Saltisford
Whitnash


In [82]:
#review the data that has been gathered

my_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bishop's Tachbrook,52.256554,-1.547717,A.C. Loyd Site,52.261854,-1.540933,Construction & Landscaping
1,Bishop's Tachbrook,52.256554,-1.547717,The Leopard,52.250665,-1.540415,Bar
2,Bishop's Tachbrook,52.256554,-1.547717,Warwick Gates,52.264805,-1.524884,Professional & Other Places
3,Bishop's Tachbrook,52.256554,-1.547717,Squab Storage,52.254257,-1.528494,Building
4,Bishop's Tachbrook,52.256554,-1.547717,Eaton. Unit 6,52.272917,-1.546374,Office


In [83]:
my_venues.shape

(2000, 7)

We can see that Venue Category contains entries which are not within scope of our chosen analysis. Here we use an inner join to filterList to remove any unwanted venue data



In [84]:
my_venues = my_venues.merge(filterList, how='inner', left_on='Venue Category', right_on='Selected')
my_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bishop's Tachbrook,52.256554,-1.547717,BMX Track Bishops Tachbrook,52.247613,-1.542722,Outdoors & Recreation
1,Bishop's Tachbrook,52.256554,-1.547717,Brakes B U13s FC,52.256613,-1.524994,Soccer Field
2,Bishop's Tachbrook,52.256554,-1.547717,Leamington Lions,52.254967,-1.524790,Soccer Field
3,Budbrooke,52.274772,-1.626225,Racing Club Warwick FC,52.275899,-1.601063,Soccer Field
4,Cubbington & Leek Wootton,52.336649,-1.512039,The Crofts,52.341840,-1.546970,Soccer Field
5,Cubbington & Leek Wootton,52.336649,-1.512039,Newbold Football Pavillion,52.308372,-1.518586,Soccer Field
6,Leamington Lillington,52.305031,-1.519627,Newbold Football Pavillion,52.308372,-1.518586,Soccer Field
7,Leamington Lillington,52.305031,-1.519627,Kingsley Playing Fields,52.308948,-1.535536,Soccer Field
8,Warwick Aylesford,52.272846,-1.601541,Racing Club Warwick FC,52.275899,-1.601063,Soccer Field
9,Whitnash,52.265666,-1.526704,Brakes B U13s FC,52.256613,-1.524994,Soccer Field


Now we can see that only venues of interest have been returned.

We need to now transform the data for clustering; specifically one-hot encoding needs to be undertaken on the venue set

In [85]:
#one hot encoding

my_venues_onehot = pd.get_dummies(my_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood back into the dataframe

my_venues_onehot['Neighborhood'] = my_venues['Neighborhood']

#reorder columns

fixed_columns = [my_venues_onehot.columns[-1]] + list(my_venues_onehot.columns[:-1])
my_venues_onehot = my_venues_onehot[fixed_columns]

my_venues_onehot

,Neighborhood,Athletics & Sports,Basketball Court,Golf Course,Gym,Gym / Fitness Center,Martial Arts Dojo,Outdoors & Recreation,Rugby Pitch,Skate Park,Soccer Field,Sports Club,Tennis Court,Yoga Studio
0,Bishop's Tachbrook,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Bishop's Tachbrook,0,0,0,0,0,0,0,0,0,1,0,0,0
2,Bishop's Tachbrook,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Budbrooke,0,0,0,0,0,0,0,0,0,1,0,0,0
4,Cubbington & Leek Wootton,0,0,0,0,0,0,0,0,0,1,0,0,0
5,Cubbington & Leek Wootton,0,0,0,0,0,0,0,0,0,1,0,0,0
6,Leamington Lillington,0,0,0,0,0,0,0,0,0,1,0,0,0
7,Leamington Lillington,0,0,0,0,0,0,0,0,0,1,0,0,0
8,Warwick Aylesford,0,0,0,0,0,0,0,0,0,1,0,0,0
9,Whitnash,0,0,0,0,0,0,0,0,0,1,0,0,0


In [86]:
#lets group the rows by neighborhood and take the frequency by category

my_venues_grouped = my_venues_onehot.groupby('Neighborhood').sum().reset_index()
my_venues_grouped

#as all the features in this data are of the same type (simple count of venues) there is no need to do feature scaling. This would not be the case if I had introduced, say, 'average rent per month GBP' into the feature set

,Neighborhood,Athletics & Sports,Basketball Court,Golf Course,Gym,Gym / Fitness Center,Martial Arts Dojo,Outdoors & Recreation,Rugby Pitch,Skate Park,Soccer Field,Sports Club,Tennis Court,Yoga Studio
0,Bishop's Tachbrook,0,0,1,0,2,1,1,0,0,2,0,0,0
1,Budbrooke,0,1,0,0,2,0,0,1,0,1,3,0,0
2,Cubbington & Leek Wootton,2,0,2,0,0,0,0,1,0,2,1,0,0
3,Kenilworth Abbey & Arden,1,0,0,0,0,0,0,1,0,0,0,0,0
4,Kenilworth Park Hill,2,0,1,0,2,0,0,0,0,0,1,1,1
5,Kenilworth St John's,0,0,0,0,1,0,0,0,0,0,0,0,0
6,Leamington Brunswick,0,0,0,0,2,0,0,0,0,0,0,0,0
7,Leamington Lillington,0,0,0,0,0,0,0,0,0,2,0,0,0
8,Leamington Milverton,0,1,0,0,0,0,0,0,0,0,0,1,0
9,Leamington Willes,0,0,0,0,2,0,0,0,0,0,0,0,0


Let's begin clustering

In [87]:
#set cluster number
kclusters = 3

#set feature set X as grouped set without neighborhood label

X = my_venues_grouped.drop('Neighborhood',1)
feature_cols = list(X.columns)

#run kmeans clustering

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(X)

#check cluster labels are being generated

kmeans.labels_[0:10]

array([0, 2, 0, 1, 2, 1, 1, 0, 1, 1], dtype=int32)

Let's now create a dataframe combining these cluster labels with the neighborhood data

In [88]:
#create new merged data dataframe and add labels

my_merged_data = my_venues_grouped
my_merged_data.insert(0, 'Cluster Labels', kmeans.labels_)


In [89]:
#recombine with original districts df to get lat and long for wards

my_merged_data  = my_merged_data.merge(right=district, how='inner', left_on='Neighborhood', right_on='Ward')

#reorder columns  by first fixing constant columns
desired_cols = ["District", "Ward", "Ward Code", "Ward_Lat", "Ward_Long", "Cluster Labels"]

#then extend the list by adding dynamically generated feature cols
desired_cols.extend(feature_cols)
my_merged_data = my_merged_data[desired_cols]

#show resulting dataframe
my_merged_data

,District,Ward,Ward Code,Ward_Lat,Ward_Long,Cluster Labels,Athletics & Sports,Basketball Court,Golf Course,Gym,Gym / Fitness Center,Martial Arts Dojo,Outdoors & Recreation,Rugby Pitch,Skate Park,Soccer Field,Sports Club,Tennis Court,Yoga Studio
0,Warwick,Bishop's Tachbrook,E05012615,52.256554,-1.547717,0,0,0,1,0,2,1,1,0,0,2,0,0,0
1,Warwick,Budbrooke,E05012616,52.274772,-1.626225,2,0,1,0,0,2,0,0,1,0,1,3,0,0
2,Warwick,Cubbington & Leek Wootton,E05012617,52.336649,-1.512039,0,2,0,2,0,0,0,0,1,0,2,1,0,0
3,Warwick,Kenilworth Abbey & Arden,E05012618,52.337111,-1.675754,1,1,0,0,0,0,0,0,1,0,0,0,0,0
4,Warwick,Kenilworth Park Hill,E05012619,52.349163,-1.564140,2,2,0,1,0,2,0,0,0,0,0,1,1,1
5,Warwick,Kenilworth St John's,E05012620,52.339397,-1.577172,1,0,0,0,0,1,0,0,0,0,0,0,0,0
6,Warwick,Leamington Brunswick,E05012621,52.278904,-1.532421,1,0,0,0,0,2,0,0,0,0,0,0,0,0
7,Warwick,Leamington Lillington,E05012623,52.305031,-1.519627,0,0,0,0,0,0,0,0,0,0,2,0,0,0
8,Warwick,Leamington Milverton,E05012624,52.296762,-1.545154,1,0,1,0,0,0,0,0,0,0,0,0,1,0
9,Warwick,Leamington Willes,E05012625,52.282296,-1.520878,1,0,0,0,0,2,0,0,0,0,0,0,0,0


### Time to create a map of our results

In [90]:
# create map
map_clusters = folium.Map(location=[myLat, myLong], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(my_merged_data['Ward_Lat'], my_merged_data['Ward_Long'], my_merged_data['Ward'], my_merged_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### How to interpret these results? Take a look at your summary data by cluster

In [91]:
summary = my_merged_data

#loop through every feature column and generate its Ward ranking, then discard original data
ranking_cols=[]
for feature in feature_cols:
    summary["{} Rank" .format(feature)] = summary[feature].rank(method='max')
    ranking_cols.append("{} Rank" .format(feature))

summary = summary.drop(feature_cols,1)


In [92]:
summary

,District,Ward,Ward Code,Ward_Lat,Ward_Long,Cluster Labels,Athletics & Sports Rank,Basketball Court Rank,Golf Course Rank,Gym Rank,Gym / Fitness Center Rank,Martial Arts Dojo Rank,Outdoors & Recreation Rank,Rugby Pitch Rank,Skate Park Rank,Soccer Field Rank,Sports Club Rank,Tennis Court Rank,Yoga Studio Rank
0,Warwick,Bishop's Tachbrook,E05012615,52.256554,-1.547717,0,10.0,13.0,13.0,13.0,15.0,15.0,15.0,11.0,13.0,15.0,10.0,13.0,14.0
1,Warwick,Budbrooke,E05012616,52.274772,-1.626225,2,10.0,15.0,10.0,13.0,15.0,13.0,14.0,15.0,13.0,11.0,15.0,13.0,14.0
2,Warwick,Cubbington & Leek Wootton,E05012617,52.336649,-1.512039,0,15.0,13.0,15.0,13.0,4.0,13.0,14.0,15.0,13.0,15.0,12.0,13.0,14.0
3,Warwick,Kenilworth Abbey & Arden,E05012618,52.337111,-1.675754,1,13.0,13.0,10.0,13.0,4.0,13.0,14.0,15.0,13.0,9.0,10.0,13.0,14.0
4,Warwick,Kenilworth Park Hill,E05012619,52.349163,-1.564140,2,15.0,13.0,13.0,13.0,15.0,13.0,14.0,11.0,13.0,9.0,12.0,15.0,15.0
5,Warwick,Kenilworth St John's,E05012620,52.339397,-1.577172,1,10.0,13.0,10.0,13.0,10.0,13.0,14.0,11.0,13.0,9.0,10.0,13.0,14.0
6,Warwick,Leamington Brunswick,E05012621,52.278904,-1.532421,1,10.0,13.0,10.0,13.0,15.0,13.0,14.0,11.0,13.0,9.0,10.0,13.0,14.0
7,Warwick,Leamington Lillington,E05012623,52.305031,-1.519627,0,10.0,13.0,10.0,13.0,4.0,13.0,14.0,11.0,13.0,15.0,10.0,13.0,14.0
8,Warwick,Leamington Milverton,E05012624,52.296762,-1.545154,1,10.0,15.0,10.0,13.0,4.0,13.0,14.0,11.0,13.0,9.0,10.0,15.0,14.0
9,Warwick,Leamington Willes,E05012625,52.282296,-1.520878,1,10.0,13.0,10.0,13.0,15.0,13.0,14.0,11.0,13.0,9.0,10.0,13.0,14.0


Create an 'average rank' to help make an overall judgement. Note that 'bigger is better'

In [93]:
summary["Average Rank"] = summary[ranking_cols].mean(axis=1)

### Ward Summary Below (Rank - Bigger Is Better)

In [94]:
#highlight top ranking cells

def highlight_max(s):
    '''
    highlight the top rank in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

summary = summary.sort_values("Average Rank", ascending=False)
summary.style.apply(highlight_max, subset=ranking_cols)


,District,Ward,Ward Code,Ward_Lat,Ward_Long,Cluster Labels,Athletics & Sports Rank,Basketball Court Rank,Golf Course Rank,Gym Rank,Gym / Fitness Center Rank,Martial Arts Dojo Rank,Outdoors & Recreation Rank,Rugby Pitch Rank,Skate Park Rank,Soccer Field Rank,Sports Club Rank,Tennis Court Rank,Yoga Studio Rank,Average Rank
1,Warwick,Budbrooke,E05012616,52.2748,-1.62622,2,10,15,10,13,15,13,14,15,13,11,15,13,14,13.1538
4,Warwick,Kenilworth Park Hill,E05012619,52.3492,-1.56414,2,15,13,13,13,15,13,14,11,13,9,12,15,15,13.1538
0,Warwick,Bishop's Tachbrook,E05012615,52.2566,-1.54772,0,10,13,13,13,15,15,15,11,13,15,10,13,14,13.0769
2,Warwick,Cubbington & Leek Wootton,E05012617,52.3366,-1.51204,0,15,13,15,13,4,13,14,15,13,15,12,13,14,13
10,Warwick,Radford Semele,E05012626,52.2917,-1.47269,2,13,13,15,15,10,13,14,11,15,9,14,13,14,13
14,Warwick,Whitnash,E05012631,52.2657,-1.5267,0,10,13,13,13,10,13,14,11,15,15,10,13,14,12.6154
12,Warwick,Warwick Aylesford,E05012628,52.2728,-1.60154,2,10,13,10,13,10,13,14,15,13,11,14,13,14,12.5385
6,Warwick,Leamington Brunswick,E05012621,52.2789,-1.53242,1,10,13,10,13,15,13,14,11,13,9,10,13,14,12.1538
9,Warwick,Leamington Willes,E05012625,52.2823,-1.52088,1,10,13,10,13,15,13,14,11,13,9,10,13,14,12.1538
11,Warwick,Warwick All Saints & Woodloes,E05012627,52.2927,-1.58535,1,13,13,10,15,10,13,14,11,13,9,10,13,14,12.1538


### Cluster Summary Below

In [95]:
cluster_summary = my_merged_data[desired_cols]
#to analyse clusters, we need to look at averages not totals
cluster_summary = cluster_summary.groupby('Cluster Labels').mean()
cluster_summary

,Ward_Lat,Ward_Long,Athletics & Sports,Basketball Court,Golf Course,Gym,Gym / Fitness Center,Martial Arts Dojo,Outdoors & Recreation,Rugby Pitch,Skate Park,Soccer Field,Sports Club,Tennis Court,Yoga Studio
Cluster Labels,,,,,,,,,,,,,,,
0,52.290975,-1.526522,0.500000,0.000000,1.00,0.000000,0.75,0.250000,0.25,0.250000,0.25,2.0,0.25,0.000000,0.00
1,52.300552,-1.570659,0.285714,0.142857,0.00,0.142857,1.00,0.142857,0.00,0.142857,0.00,0.0,0.00,0.142857,0.00
2,52.297127,-1.566148,0.750000,0.250000,0.75,0.250000,1.50,0.000000,0.00,0.500000,0.25,0.5,2.00,0.250000,0.25


In [96]:
#loop through every feature column and generate its Ward ranking, then discard original data

for feature in feature_cols:
    cluster_summary["{} Rank" .format(feature)] = cluster_summary[feature].rank(method='max')
    

cluster_summary = cluster_summary.drop(feature_cols,1)

In [101]:
cluster_summary["Average Rank"] = cluster_summary[ranking_cols].mean(axis=1)

cluster_summary.style.apply(highlight_max, subset=ranking_cols)

,Ward_Lat,Ward_Long,Athletics & Sports Rank,Basketball Court Rank,Golf Course Rank,Gym Rank,Gym / Fitness Center Rank,Martial Arts Dojo Rank,Outdoors & Recreation Rank,Rugby Pitch Rank,Skate Park Rank,Soccer Field Rank,Sports Club Rank,Tennis Court Rank,Yoga Studio Rank,Average Rank
Cluster Labels,,,,,,,,,,,,,,,,
0,52.291,-1.52652,2,1,3,1,1,3,3,2,3,3,2,1,2,2.07692
1,52.3006,-1.57066,1,2,1,2,2,2,2,1,1,1,1,2,2,1.53846
2,52.2971,-1.56615,3,3,2,3,3,1,2,3,3,2,3,3,3,2.61538


# Results

Let's group the outputs for ease of reading - top performing groups are highlighted in green for your features.

## Ward Level Analysis

Your preferred Ward may not have the highest Average Rank - consider the feature-level rankings too

In [98]:
summary.style.apply(highlight_max, subset=ranking_cols)

,District,Ward,Ward Code,Ward_Lat,Ward_Long,Cluster Labels,Athletics & Sports Rank,Basketball Court Rank,Golf Course Rank,Gym Rank,Gym / Fitness Center Rank,Martial Arts Dojo Rank,Outdoors & Recreation Rank,Rugby Pitch Rank,Skate Park Rank,Soccer Field Rank,Sports Club Rank,Tennis Court Rank,Yoga Studio Rank,Average Rank
1,Warwick,Budbrooke,E05012616,52.2748,-1.62622,2,10,15,10,13,15,13,14,15,13,11,15,13,14,13.1538
4,Warwick,Kenilworth Park Hill,E05012619,52.3492,-1.56414,2,15,13,13,13,15,13,14,11,13,9,12,15,15,13.1538
0,Warwick,Bishop's Tachbrook,E05012615,52.2566,-1.54772,0,10,13,13,13,15,15,15,11,13,15,10,13,14,13.0769
2,Warwick,Cubbington & Leek Wootton,E05012617,52.3366,-1.51204,0,15,13,15,13,4,13,14,15,13,15,12,13,14,13
10,Warwick,Radford Semele,E05012626,52.2917,-1.47269,2,13,13,15,15,10,13,14,11,15,9,14,13,14,13
14,Warwick,Whitnash,E05012631,52.2657,-1.5267,0,10,13,13,13,10,13,14,11,15,15,10,13,14,12.6154
12,Warwick,Warwick Aylesford,E05012628,52.2728,-1.60154,2,10,13,10,13,10,13,14,15,13,11,14,13,14,12.5385
6,Warwick,Leamington Brunswick,E05012621,52.2789,-1.53242,1,10,13,10,13,15,13,14,11,13,9,10,13,14,12.1538
9,Warwick,Leamington Willes,E05012625,52.2823,-1.52088,1,10,13,10,13,15,13,14,11,13,9,10,13,14,12.1538
11,Warwick,Warwick All Saints & Woodloes,E05012627,52.2927,-1.58535,1,13,13,10,15,10,13,14,11,13,9,10,13,14,12.1538


## Cluster Level Analysis

Your top ranked cluster may be the best performing overall - however you may PREFER wards of a different cluster

In [99]:
cluster_summary.style.apply(highlight_max, subset=ranking_cols)

,Ward_Lat,Ward_Long,Athletics & Sports Rank,Basketball Court Rank,Golf Course Rank,Gym Rank,Gym / Fitness Center Rank,Martial Arts Dojo Rank,Outdoors & Recreation Rank,Rugby Pitch Rank,Skate Park Rank,Soccer Field Rank,Sports Club Rank,Tennis Court Rank,Yoga Studio Rank,Average Rank
Cluster Labels,,,,,,,,,,,,,,,,
0,52.291,-1.52652,2,1,3,1,1,3,3,2,3,3,2,1,2,2.07692
1,52.3006,-1.57066,1,2,1,2,2,2,2,1,1,1,1,2,2,1.53846
2,52.2971,-1.56615,3,3,2,3,3,1,2,3,3,2,3,3,3,2.61538


## Your Map

In [100]:
map_clusters